In [1]:
import pandas as pd
from datetime import datetime
import re

# Read the CSV file into a DataFrame
df = pd.read_csv("output.csv")

# Get the number of rows in the DataFrame
num_rows = len(df)

# Calculate the index to split the DataFrame into two halves
split_index = num_rows // 2

# Take only the second half of the DataFrame
df = df.iloc[split_index-1:]
df.drop(columns=['Unnamed: 5'], inplace=True)

new_column_names = ['PredictedArrival', 'Flight Id','Destination','AirLine','Aircraft','Status']  # Specify your new column names
df.columns = new_column_names
df.reset_index(drop=True, inplace=True)



In [2]:
df["PredictedArrival"] = df["PredictedArrival"].astype(str)
print(df)



                      PredictedArrival Flight Id           Destination  \
0                                16:38    AA2941          Tampa (TPA)-   
1                                16:40    AA2620    Montego Bay (MBJ)-   
2                                16:40    UA1030       New York (EWR)-   
3                                16:44    AA2293        Norfolk (ORF)-   
4                                16:48     AA487  San Francisco (SFO)-   
..                                 ...       ...                   ...   
515                              11:12    AA1661      Cleveland (CLE)-   
516                              11:12    NK1699        Orlando (MCO)-   
517                              11:15    AA1834   Jacksonville (JAX)-   
518                 Load later flights       NaN                   NaN   
519  * All times are in local timezone       NaN                   NaN   

                 AirLine       Aircraft        Status  
0    American Airlines -  A321 (N903AA)  Landed 16:25  

In [3]:

# Define a regular expression pattern to match days
day_pattern = r"(\b(?:Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\b)"

# Get the current year
current_year = datetime.now().year

# Filter rows that match the day format
day_rows = df[df['PredictedArrival'].str.match(day_pattern, na=False)]

# Iterate over the indices of rows with day format
for idx in day_rows.index:
    # Get the value in the 'Departure' column
    value = df.at[idx, 'PredictedArrival']
    
    # Split the value using the comma and space as separators
    day_month_year = value.split(', ')[-1].split()

    # Extract day, month, and year
    day = day_month_year[1]
    month = datetime.strptime(day_month_year[0], '%b').strftime('%m')  # Convert month to numerical format

    # Transform the value to the desired format
    transformed_value = f"{day}/{month}/{current_year}"

    # Update the value in the DataFrame
    df.at[idx, 'PredictedArrival'] = transformed_value

# Print the updated DataFrame
print(df)

                      PredictedArrival Flight Id           Destination  \
0                                16:38    AA2941          Tampa (TPA)-   
1                                16:40    AA2620    Montego Bay (MBJ)-   
2                                16:40    UA1030       New York (EWR)-   
3                                16:44    AA2293        Norfolk (ORF)-   
4                                16:48     AA487  San Francisco (SFO)-   
..                                 ...       ...                   ...   
515                              11:12    AA1661      Cleveland (CLE)-   
516                              11:12    NK1699        Orlando (MCO)-   
517                              11:15    AA1834   Jacksonville (JAX)-   
518                 Load later flights       NaN                   NaN   
519  * All times are in local timezone       NaN                   NaN   

                 AirLine       Aircraft        Status  
0    American Airlines -  A321 (N903AA)  Landed 16:25  

In [4]:
# Define a regular expression pattern to match the desired format
date_pattern = r"\d{2}/\d{2}/\d{4}"

# Initialize a list to store the indices
indices = []

# Iterate over the DataFrame and check each value in the "Departure" column
for idx, value in df['PredictedArrival'].items():
    # Check if the value matches the desired format
    if isinstance(value, str) and any(substring in value for substring in ['/']):
        # If it matches, append the index to the list
        indices.append(idx)

# Print the indices
print("Indices of rows with the format dd/mm/yyyy:", indices)

Indices of rows with the format dd/mm/yyyy: [275]


In [7]:
# Initialize an empty list to store the new column values
new_column_values = []

# Iterate over the indices
for i, index in enumerate(indices):
    print("Current index:", index)  # Debugging print
    # Get the end index for the current range
    end_idx = indices[i + 1] if i < len(indices) - 1 else len(df)
    print("End index:", end_idx)  # Debugging print
    
    # Extract the value from the "Departure" column for the current range
    value = df.loc[index, "PredictedArrival"]
    print("Value:", value)  # Debugging print
    
    # Append the value to the new_column_values list for each row in the current range
    rows_count = end_idx - index
    print("Rows count:", rows_count)  # Debugging print
    new_column_values.extend([value] * rows_count)

# Add the new column to the DataFrame
df['Date'] = new_column_values

# Display the updated DataFrame
print(df)


Current index: 275
End index: 520
Value: 26/02/2024
Rows count: 245


ValueError: Length of values (245) does not match length of index (520)

In [ ]:
# Split the "Status" column into two columns using space separator
df[['Status', 'TimeOfLanding']] = df['Status'].str.split(' ', n=1, expand=True)
df.drop(columns=['Status'], inplace=True)

# Remove rows with NaN values
df = df.dropna()
df.reset_index(drop=True, inplace=True)
# Display the updated DataFrame
print(df)


In [ ]:
df.to_csv('dfoutput.csv', index=False, encoding='utf-8', header=False)

In [ ]:
column_types = df.dtypes

# Print the data types of each column
print(column_types)